In [1]:
import scanpy as sc 
import anndata
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import os

import datetime


In [2]:
#Define constants here 
#Current date format is YYYYMMDD with automatically updated date
today = datetime.datetime.now().strftime("%Y%m%d")

In [3]:
def read_10x_output(file_path, smp_list, metadata=None, type = 'raw', umi_filter=5):
    import os
    
    #Writing output from separate samples, processed using CellRanger, into a dictionary of Scanpy objects:
    ad = {}

    #Generate AnnData for each sample
    for sample_name in smp_list:
        path = file_path + sample_name
        for i in os.listdir(path):
            if type in i and 'h5' in i:
                file = i
        ad[sample_name] = sc.read_10x_h5(file_path + sample_name +'/'+file)
        ad[sample_name].var.rename(columns = {'gene_ids':'ENSEMBL'}, inplace = True)
        ad[sample_name].var['SYMBOL'] = ad[sample_name].var.index
        ad[sample_name].var.index = ad[sample_name].var['ENSEMBL']
        ad[sample_name].var.drop(columns=['ENSEMBL'], inplace=True)
        #ad[sample_name].var_names_make_unique() 
        
        
        sc.pp.calculate_qc_metrics(ad[sample_name], inplace=True)
        ad[sample_name] = ad[sample_name][ad[sample_name].obs['total_counts'] > umi_filter, :]
        ad[sample_name].var['mt'] = [gene.startswith('mt-') 
                                     for gene in ad[sample_name].var['SYMBOL']]
        ad[sample_name].obs['mt_frac'] = (ad[sample_name][:, 
               ad[sample_name].var['mt'].tolist()].X.sum(1).A.squeeze() 
                                          / ad[sample_name].obs['total_counts'])
        
        ad[sample_name].obs['sample'] = sample_name
        ad[sample_name].obs['barcode'] = ad[sample_name].obs_names
        ad[sample_name].obs_names = ad[sample_name].obs['sample']+"_"+ad[sample_name].obs['barcode']

    #Merge AnnData objects from all the samples together    
    from scipy.sparse import vstack
    stack = vstack([ad[x].X for x in smp_list]) # stack data
    adata = sc.AnnData(stack, var = ad[smp_list[0]].var)
    adata.obs = pd.concat([ad[x].obs for x in smp_list], axis = 0)

    if metadata is not None:
        #Add cleaned metadata to the Anndata.obs table
        obs_merged = pd.merge(left = adata.obs, right = metadata, 
                              how = "left", left_on="sample", right_on="sample")
        obs_merged.index = obs_merged['sample']+"_"+obs_merged['barcode']
        print(obs_merged.index.equals(adata.obs.index))
        adata.obs = obs_merged

    return adata, ad


In [4]:
#Create a sample list in form of a dataframe to be concatenated by the read_10x_output function
sample_list = pd.DataFrame({
    'sample': ['cellranger710_count_05e682d5679826b9b76d6bec731bbe61']
})

In [6]:
#Save results in the following folder (create if it doesn't exist)

results_folder = '/lustre/scratch123/hgi/teams/parts/kl11/cell2state_tf_activation/results/'
if not os.path.exists(results_folder):
    os.makedirs(results_folder)
    
sc_data_folder = '/nfs/team205/vk7/sanger_projects/cell2state_tf_activation/data/pilot_5_prime_dec_2022/'

#Read in 10x data in raw and filtered format
adata, ad_list = read_10x_output(
    sc_data_folder, 
    smp_list=sample_list['sample'],
    metadata=sample_list, type = 'raw')
# export aggregated and pre-processed data
adata.write(f'{results_folder}{today}_all_cells_with_empty.h5ad')

adata, ad_list = read_10x_output(
    sc_data_folder, 
    smp_list=sample_list['sample'],
    metadata=sample_list, type = 'filtered')
# export aggregated and pre-processed data
adata.write(f'{results_folder}{today}_all_cells.h5ad')

/nfs/team205/kl11/miniconda3/envs/bcftools/lib/python3.9/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/tmp/ipykernel_11071/435707439.py:23: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  ad[sample_name].var['mt'] = [gene.startswith('mt-')


True


/nfs/team205/kl11/miniconda3/envs/bcftools/lib/python3.9/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/tmp/ipykernel_11071/435707439.py:23: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  ad[sample_name].var['mt'] = [gene.startswith('mt-')


True


In [9]:
adata.obs

,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,mt_frac,sample,barcode
cellranger710_count_05e682d5679826b9b76d6bec731bbe61_AAACCTGAGAATCTCC-1,6028,8.704336,19097.0,9.857339,17.971409,25.878410,34.827460,47.677646,0.0,cellranger710_count_05e682d5679826b9b76d6bec73...,AAACCTGAGAATCTCC-1
cellranger710_count_05e682d5679826b9b76d6bec731bbe61_AAACCTGAGAGTACCG-1,8290,9.022926,47036.0,10.758690,20.665023,29.698529,38.774556,51.724211,0.0,cellranger710_count_05e682d5679826b9b76d6bec73...,AAACCTGAGAGTACCG-1
cellranger710_count_05e682d5679826b9b76d6bec731bbe61_AAACCTGAGCTAGTTC-1,5973,8.695172,29295.0,10.285206,23.604711,33.947773,43.894863,58.088411,0.0,cellranger710_count_05e682d5679826b9b76d6bec73...,AAACCTGAGCTAGTTC-1
cellranger710_count_05e682d5679826b9b76d6bec731bbe61_AAACCTGAGTGGTAGC-1,7235,8.886824,33495.0,10.419181,19.358113,27.902672,36.886102,49.720854,0.0,cellranger710_count_05e682d5679826b9b76d6bec73...,AAACCTGAGTGGTAGC-1
cellranger710_count_05e682d5679826b9b76d6bec731bbe61_AAACCTGCACGGATAG-1,6792,8.823648,30050.0,10.310651,20.569052,29.906822,39.021631,52.259567,0.0,cellranger710_count_05e682d5679826b9b76d6bec73...,AAACCTGCACGGATAG-1
...,...,...,...,...,...,...,...,...,...,...,...
cellranger710_count_05e682d5679826b9b76d6bec731bbe61_TTTGTCAGTTTGTTGG-1,6645,8.801770,22523.0,10.022337,17.559828,25.476180,33.836523,46.401456,0.0,cellranger710_count_05e682d5679826b9b76d6bec73...,TTTGTCAGTTTGTTGG-1
cellranger710_count_05e682d5679826b9b76d6bec731bbe61_TTTGTCATCACCTCGT-1,5836,8.671972,18445.0,9.822603,18.400651,26.928707,35.543508,48.479263,0.0,cellranger710_count_05e682d5679826b9b76d6bec73...,TTTGTCATCACCTCGT-1
cellranger710_count_05e682d5679826b9b76d6bec731bbe61_TTTGTCATCTACCAGA-1,4994,8.516193,16715.0,9.724122,21.543524,31.379001,40.873467,54.316482,0.0,cellranger710_count_05e682d5679826b9b76d6bec73...,TTTGTCATCTACCAGA-1
cellranger710_count_05e682d5679826b9b76d6bec731bbe61_TTTGTCATCTATGTGG-1,5933,8.688454,17312.0,9.759212,17.074861,24.740065,32.798059,45.563771,0.0,cellranger710_count_05e682d5679826b9b76d6bec73...,TTTGTCATCTATGTGG-1


In [7]:
def subset_cells(
    adata,
    cells_per_category=10000,
    stratify_category_key='sample',
):
    
    adata.obs['_cell_index'] = np.arange(adata.n_obs)
    subset_ind = list()
    
    for ct in adata.obs[stratify_category_key].unique():
        ind = adata.obs[stratify_category_key] == ct
        subset_ind_ = adata.obs['_cell_index'][ind]
        n_samples = np.min((len(subset_ind_), cells_per_category))
        subset_ind = subset_ind + list(np.random.choice(subset_ind_, size=n_samples, replace=False))
    print(len(subset_ind))
    
    return adata[subset_ind, :].copy()

def compute_pcs_knn_umap(
    adata_subset, stratify_category_key='sample', 
    tech_category_key=None, plot_category_keys=list(), 
    scale_max_value=10, n_comps=100, n_neighbors=15,
):
    adata_subset.obs['total_counts'] = np.array(adata_subset.X.sum(1)).flatten()
    adata_subset.layers['counts'] = adata_subset.X.copy()
    # No normalisation by total count
    sc.pp.log1p(adata_subset)
    # Scale with no HVG selection
    if tech_category_key is None:
        sc.pp.scale(adata_subset, max_value=scale_max_value)
    else:
        for tech in adata_subset.obs[tech_category_key].unique():
            mu, std = compute_mu_std(adata_subset[adata_subset.obs[tech_category_key] == tech].X)
            adata_subset[adata_subset.obs[tech_category_key] == tech].X = (
                np.minimum((adata_subset[adata_subset.obs[tech_category_key] == tech].X - mu) / std, scale_max_value)
            )
    # A lot of PC dimensions
    sc.tl.pca(adata_subset, svd_solver='arpack', n_comps=n_comps, use_highly_variable=False)
    # Plot PCs to confirm that PC1 is indeed linked to total count
    # sc.pl.pca(adata_subset, color=['total_counts'],
    #           components=['1,2', '2,3', '4,5'],
    #           color_map = 'RdPu', ncols = 3, legend_loc='on data',
    #           legend_fontsize=10)
    plt.hist2d(adata_subset.obsm['X_pca'][:, 0].flatten(),
               adata_subset.obs['total_counts'].values.flatten(),
               bins=200,
               norm=mpl.colors.LogNorm());
    plt.xlabel('PC 1');
    plt.ylabel('Total RNA count');
    plt.show()

    # Remove PC1
    adata_subset.obsm['X_pca'] = adata_subset.obsm['X_pca'][:, 1:]
    adata_subset.varm['PCs'] = adata_subset.varm['PCs'][:, 1:]

    # compute KNN and UMAP to see how well this represents the dataset
    sc.pp.neighbors(adata_subset, n_neighbors=n_neighbors)
    sc.tl.umap(adata_subset, min_dist = 0.4, spread = 1.5)

    # Plot UMAP
    sc.pl.umap(adata_subset, color=[stratify_category_key] + plot_category_keys,
               color_map = 'RdPu', ncols = 3, #legend_loc='on data',
               legend_fontsize=10)
    return adata_subset

def compute_mu_std(X):
        
    mu = np.array(X.mean(0))
    mu_sq = mu ** 2
    X = X.copy()
    X.data = X.data ** 2
    sq_mu = np.array(X.mean(0))
    std = np.sqrt(sq_mu - mu_sq) + 1e-8
    
    return mu, std